In [84]:
!pip install openai langchain nest_asyncio playwright html2text sentence-transformers qdrant-client requests
!pip install groq python-dotenv langchain-groq langchain-openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 907.0/907.0 kB 2.2 MB/s eta 0:00:00a 0:00:01


In [2]:
#Only if first time running playwright on the machine
#This will install chromium-compatible browser
!playwright install

## Prepare your Data

In [2]:
# Load Environment variables
from dotenv import load_dotenv
import os

load_dotenv()

True

### 1. Load data from the web

In [74]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.document_loaders import AsyncChromiumLoader
from langchain.document_transformers import Html2TextTransformer

def scrape_documents_from_web():
    # articles = ["https://react.dev/blog/2024/04/25/react-19"]
    articles = [
        "https://www.uefa.com/uefachampionsleague/news/028d-1af2b2b6fa83-105b56c607b6-1000--champions-league-final-preview-dortmund-vs-real-madrid-wh/"
    ]

    # Scrapes the web pages using playwright
    loader = AsyncChromiumLoader(articles)
    docs = loader.load()

    # Converts to plain text
    html2text = Html2TextTransformer()
    docs_transformed = html2text.transform_documents(docs)

    # Chunk text
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )
    chunked_documents = text_splitter.split_documents(docs_transformed)

    return chunked_documents

### 2. Load Data from Perigon

In [56]:
import requests 
from langchain.docstore.document import Document

def get_documents_from_perigon():
    url = "https://api.goperigon.com/v1/all?q='Dortmund vs Real Madrid 2024' AND 'Dortmund vs Real Madrid line-ups'"

    payload = {}
    headers = {"x-api-key": os.environ.get("PERIGON_API_KEY")}

    response = requests.request("GET", url, headers=headers, data=payload)

    articles_transformed = [article["content"] for article in response.json()["articles"]]
    docs_transformed = [
        Document(page_content=article_content)
        for article_content in articles_transformed
    ]
    print(docs_transformed)

    # Chunk text
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )
    chunked_documents = text_splitter.split_documents(docs_transformed)

    print(chunked_documents)
    return chunked_documents

In [30]:
get_documents_from_perigon()

[Document(page_content="React 18 was announced in March 2022 and with this many new and amazing features were added. For instance, automatic batching wasn’t available in the previous versions. But in React 18, it was added. And now, as per the React blog, React 19 is also coming.\n\nManual memoization is done whenever there’s too much re-rendering in case a state changes. But you see this very action is not right. Because it can make your code a bit messy. Just one wrong step and you’ll go down the rabbit hole. Even React considers manual memoization just a compromise.\n\n They want React to re-render itself whenever there’s a state change. So, now they are developing a React compiler. It will prevent unnecessary renders. Instagram is already using it in its production. But what’s the advantage of React compiler? Well, it renders the right parts wherever a state is changed. In fact, it will also compile code as per the rules of JavaScript and React. In other words, this means that it w

[Document(page_content='React 18 was announced in March 2022 and with this many new and amazing features were added. For instance, automatic batching wasn’t available in the previous versions. But in React 18, it was added. And now, as per the React blog, React 19 is also coming.\n\nManual memoization is done whenever there’s too much re-rendering in case a state changes. But you see this very action is not right. Because it can make your code a bit messy. Just one wrong step and you’ll go down the rabbit hole. Even React considers manual memoization just a compromise.\n\n They want React to re-render itself whenever there’s a state change. So, now they are developing a React compiler. It will prevent unnecessary renders. Instagram is already using it in its production. But what’s the advantage of React compiler? Well, it renders the right parts wherever a state is changed. In fact, it will also compile code as per the rules of JavaScript and React. In other words, this means that it w

## Embed Documents Into the Vector Database

In [75]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

import nest_asyncio

nest_asyncio.apply()

# Get Documents from the Web
docs_from_web = scrape_documents_from_web()
# Or Get Documents using API providers, like Perigon
docs_from_perigon = get_documents_from_perigon()

# Load chunked documents into the Qdrant vector DB
db = Qdrant.from_documents(
    docs_from_web,
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"),
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

# Connect the retriever to the Qdrant DB
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Document(page_content='The best of the best will be crowned on the ultimate stage when the 2023/24 UEFA Champions League season concludes at London\'s prestigious Wembley Stadium on Saturday 1 June.\n\nThis is the 69th season of Europe\'s elite club competition and the 32nd since it was renamed the UEFA Champions League.\n\nWho is playing in the Champions League final?\n\nBorussia Dortmund were first to confirm their place in the final with a 2-0 aggregate victory over Paris Saint-Germain. They will take on 14-time champions Real Madrid, who secured their own ticket to London with a dramatic 4-3 aggregate victory over Bayern München.\n\nWhere is the Champions League final?\n\nThe highlight of European football\'s club calendar returns to the London venue for a record eighth time, and the third in the UEFA Champions League era following the 2011 and 2013 finals.\n\nWembley has undergone a massive transformation since it hosted the last old-style European Champion Clubs\' Cup final in 1

/Users/islemmaboud/.pyenv/versions/3.11.3/envs/mlenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Initialize your LLM Model of Choice

In [32]:
from langchain_community.llms import Ollama
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_openai import OpenAI

# If you wish to use Ollama
llama3_llm = Ollama(model="llama3-8b-instruct")
# If you wish to use Groq
groq_llama3_llm = ChatGroq(
    temperature=0.4,
    groq_api_key=os.environ.get("GROQ_API_KEY"),
    model_name="llama3-8b-8192",
)
# If you wish to use OpenAI ChatGPT
openai_llm = OpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"))

## Testing Prompt: Without RAG (Using Directly The LLM) ❌

In [62]:
##################
## Without RAG ❌
#################

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | groq_llama3_llm
query = "Where to watch, match date and possible line-ups for Dortmund vs Real finale"

for s in chain.stream(query):
    # print(s)
    print(s.content, end="", flush=True)
    # display_markdown(s.content, raw=True)

The highly anticipated UEFA Champions League Final between Borussia Dortmund and Real Madrid!

Here's the essential information you need to know:

**Match Details:**

* Date: Saturday, May 28, 2023
* Kick-off time: 21:00 CEST (Central European Summer Time)
* Venue: Atatürk Olympic Stadium, Istanbul, Turkey

**Where to Watch:**

The UEFA Champions League Final will be broadcast in many countries around the world. You can check the official UEFA website for a list of broadcasters and streaming options in your region.

In the United States, you can watch the match on CBS Sports Network and stream it on Paramount+. In the UK, it will be broadcast on BT Sport 2 and streamed on the BT Sport app.

**Possible Line-ups:**

Please note that these line-ups are subject to change and may not reflect the actual starting XI for the match. Here are some possible line-ups for both teams:

**Borussia Dortmund:**

* Goalkeeper: Gregor Kobel
* Defenders: Nico Schulz, Mats Hummels, Manuel Akanji, Thomas Me

## Preparing Prompt Template for Providing Context to the LLM Model 

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough

# Prepare Prompt template to include the retrieved context 
# for the user prompt

prompt_template = """
### [INST] 
Instruction: Answer the question based on your latest Knowledge

{context}

### QUESTION:
{question} 

[/INST]
"""

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=groq_llama3_llm, prompt=prompt)

/Users/islemmaboud/.pyenv/versions/3.11.3/envs/mlenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


## Testing Prompt: With RAG ✅

In [78]:
###############
## With RAG ✅
##############

from IPython.display import display_markdown

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

# query = "how can I use useActionState hook?"
# query = (`
#     "Give me an example of using the new useActionState hook for a login form"
# )`

query = "Where to watch, match date and possible line-ups for Dortmund vs Real Madrid final"

for s in rag_chain.stream(query):
    # print(s['text'], end="", flush=True)
    display_markdown(s['text'], raw=True) 

According to the article, here are the answers to your question:

**Where to watch the Champions League final:** Fans can find their local UEFA Champions League broadcast partner(s) here.

**Match date:** Saturday 1 June.

**Possible line-ups:**

**Dortmund:** Kobel; Ryerson, Hummels, Schlotterbeck, Maatsen; Can, Sabitzer; Adeyemi, Brandt, Sancho; Füllkrug

**Real Madrid:** Courtois; Carvajal, Éder Militão, Rüdiger, Mendy; Valverde, Kroos; Camavinga; Bellingham; Rodrygo, Vinícius Júnior